In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
#ee.Authenticate()
#geemap.update_package()

ee.Initialize()

Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

##Adding every plot coordinate
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'
fc_all = geemap.csv_to_ee(allplots_fc, latitude = "Latitude", longitude = "Longitude")

In [4]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.

#IS THIS SAME BETWEEN SENSORS: https://github.com/giswqs/geemap/discussions/672

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)

#NOTE 10/6/2022 - This has been updated for Landsat Collection 2 https://www.usgs.gov/landsat-missions/landsat-collection-2

In [5]:
##TIDAL FILTERING; from Narron et al. 2022
##Utilizes L8 bands 4 and 6 for NDWI, and bands 3 and 4 (for pheno)
##Does it work for Landsat 5?

# def addFLATS(image):
#     flats = ee.Image(0).expression(
#         '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
#             'SWIR': image.select('SR_B6'),
#             'RED': image.select('SR_B4'),
#             'GREEN': image.select('SR_B3')
#         })
    
#     return image.addBands(flats.rename('flats'))

# def addFLATSL5(image):
#     flats = ee.Image(0).expression(
#         '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
#             'SWIR': image.select('SR_B5'),
#             'RED': image.select('SR_B3'),
#             'GREEN': image.select('SR_B2')
#         })
    
#     return image.addBands(flats.rename('flats'))

#Updated 10/6/2022 - Band names changed for Collection 2


#12/08/22 - adding CALIBRATED FLATS
def addFLATSL7(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(RED-SWIR)/(RED+SWIR) - 41.2*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.972*(RED-SWIR)/(RED+SWIR)-0.008) - 41.2*(0.991*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)-0.0014)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.rename('flats'))


def addFLATSL8(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.841*(RED-SWIR)/(RED+SWIR) - 0.019) - 41.2*(0.771*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1) + 0.011)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL9(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(1.225*(RED-SWIR)/(RED+SWIR) + 0.096) - 41.2*(1.038* (NIR-RED)/(NIR+6*RED-7.5*BLUE+1) - 0.004)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

##MASKING FLATS
def maskFLATS(image):
    mask1 = image.select('flats').lte(0.1) #less than or equal to 0.1 - change?
    return image.updateMask(mask1)

##ADDING NDVI (for min/max variables)
def addL5ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'GREEN': image.select('SR_B2')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

def addL8ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'GREEN': image.select('SR_B3')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

In [6]:
##Pixel extraction functions - addDate for dateless images/collections
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

##For Landsat images:
def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 30,
        tileScale = 16 #ADDED 10/6/2022 - make sure it doesn't affect results (see thread below)
    )
    return feature

##FOR 10m DEM:
def demExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 10 
    )
    return feature

##FOR 1m DEM:
def dem1Extraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 1, 
        tileScale = 16 #ADDED 11/7/2022

    )
    return feature

#tileScale: https://gis.stackexchange.com/questions/373250/understanding-tilescale-in-earth-engine

In [7]:
##Adding DEM
dem = ee.Image('USGS/3DEP/10m') ##This is 1/3 arc second, or 10 m.
dem1 = ee.ImageCollection('USGS/3DEP/1m')

##Set visualization parameters.
dem_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, dem_params, '10m DEM')
Map.addLayer(dem1, dem_params, '1m DEM')

In [8]:
##Calculating mean pixel values for time periods within each year

##NOTE (11/7/22): got 1 m elevation extraction working, and want to consolidate all variables onto one df

def monthly_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

##Months and years are lists

years_ls5 = range(2000, 2012)
years_ls7 = range(2012, 2013)
years_ls8 = range(2013, 2021)
years_dm = range(2000, 2021)

months = range(5,11)
months_daymet = range(1,12)

ls5_collect = ee.ImageCollection(
    'LANDSAT/LT05/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATSL5).map(maskFLATS).map(addL5ndvi)

ls7_collect = ee.ImageCollection(
    'LANDSAT/LE07/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATSL7).map(maskFLATS).map(addL5ndvi) ####SHOULD BE addFLATSL5

#Note:Landsat datasets migrated to Collection 2

ls8_collect = ee.ImageCollection(
    'LANDSAT/LC08/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATSL8).map(maskFLATS).map(addL8ndvi)

monthly_ls5 = monthly_Avg(ls5_collect, years = years_ls5)
monthly_ls7 = monthly_Avg(ls7_collect, years = years_ls7)
monthly_ls8 = monthly_Avg(ls8_collect, years = years_ls8)

monthly_ls5.size().getInfo()

##From stackexchange: calculating monthly averages across many years:
#https://gis.stackexchange.com/questions/290892/google-earth-enginesst-by-month-per-year
#https://gis.stackexchange.com/questions/426662/image-collection-monthly-averages-using-geemap-package

12

In [9]:
def year_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


year_ls5 = year_Avg(ls5_collect, years = years_ls5)
year_ls7 = year_Avg(ls7_collect, years = years_ls7)
year_ls8 = year_Avg(ls8_collect, years = years_ls8)

yearlist_5 = year_ls5.toList(year_ls5.size())
yearlist_7 = year_ls7.toList(year_ls7.size())
yearlist_8 = year_ls8.toList(year_ls8.size())

def peak_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(8, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

peak_ls5 = peak_Avg(ls5_collect, years = years_ls5)
peak_ls7 = peak_Avg(ls7_collect, years = years_ls7)
peak_ls8 = peak_Avg(ls8_collect, years = years_ls8)

peaklist_5 = peak_ls5.toList(peak_ls5.size())
peaklist_7 = peak_ls7.toList(peak_ls7.size())
peaklist_8 = peak_ls8.toList(peak_ls8.size())

In [10]:
list_5 = monthly_ls5.toList(monthly_ls5.size())
list_7 = monthly_ls7.toList(monthly_ls7.size())
list_8 = monthly_ls8.toList(monthly_ls8.size())

vis_param = {'min': 0, 
             'max': 0.2, 
             'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 
             'gamma': 1.5}

x= ee.Image(list_5.get(0))
y= ee.Image(list_8.get(0))

Map.addLayer(x, vis_param)

Map.addLayer(fc_all)
Map

Map(center=[31.539096, -81.422318], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

In [11]:
#Daymet
def dm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def peakdm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(8, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filterBounds(fc_all)

#Over year
monthly_dm = dm_Avg(daymet, years = years_dm)
dm_list = monthly_dm.toList(monthly_dm.size())

#Peak biomass
peak_dm = peakdm_Avg(daymet, years = years_dm)
peakdm_list = peak_dm.toList(peak_dm.size())

In [12]:
#Elevation
dem_vals = geemap.ee_to_pandas(demExtraction(dem)) ##10m dataset
# dem_vals = geemap.ee_to_pandas(dem1.map(dem1Extraction).flatten()) ##ONE METER DATASET

In [13]:
#With loop to cut out unnecessary code:
years_l5 = range(2000, 2012)
years_l7 = range(2012, 2013)
years_l8 = range(2013, 2021)

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(list_5.get(i))))
    sample5 = ls5_x[ls5_x['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(list_7.get(i))))
    sample7 = ls7_x[ls7_x['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(list_8.get(i))))
    sample8 = ls8_x[ls8_x['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)

landsat_extract = pd.concat([l5_extract,l7_extract,l8_extract])

landsat_extract

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,...,QA_PIXEL,ST_CDIST,flats,SR_B1,ST_QA,ST_DRAD,ndvi,SR_QA_AEROSOL,SR_B6,ST_B10
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,...,5440.0,325.833333,0.016652,0.051730,414.333333,1608.333333,0.369552,NaN,NaN,NaN
19,2,A1,2000,1,1,31.536199,1152.400,21,-81.296118,10/11/2000,...,5465.6,179.700000,0.015116,0.046829,456.200000,1446.600000,0.390318,NaN,NaN,NaN
21,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,...,5491.2,116.400000,0.058649,0.054947,492.400000,1529.400000,0.334554,NaN,NaN,NaN
33,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,...,5440.0,204.769231,0.006469,0.043557,432.461538,1347.846154,0.444670,NaN,NaN,NaN
52,6,A1,2000,1,1,31.388423,1032.632,20,-81.279792,10/12/2000,...,5440.0,59.300000,0.037714,0.057843,476.500000,1501.100000,0.332405,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154,7,A4,2019,2,8,31.334597,2365.620,14,-81.478039,10/8/2019,...,21824.0,95.375000,0.006179,0.011815,472.125000,1914.875000,0.620638,200.50,0.079929,302.723313
3160,7,A4,2019,1,12,31.335017,1821.010,12,-81.478711,10/8/2019,...,21824.0,108.375000,0.000853,0.014710,421.625000,1820.500000,0.670045,180.25,0.110234,302.394329
3164,7,A4,2019,1,13,31.334934,2482.584,14,-81.478709,10/8/2019,...,21824.0,108.375000,0.003180,0.014768,423.500000,1820.500000,0.615541,176.50,0.086199,302.487043
3168,7,A4,2019,1,16,31.334682,2511.936,17,-81.478564,10/8/2019,...,21824.0,108.375000,0.003180,0.014768,423.500000,1820.500000,0.615541,176.50,0.086199,302.487043


In [14]:
#Yearly:
years_l5 = range(2000, 2012)
years_l7 = range(2012, 2013)
years_l8 = range(2013, 2021)

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(yearlist_5.get(i))))
    sample5 = ls5_x[ls5_x['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(yearlist_7.get(i))))
    sample7 = ls7_x[ls7_x['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(yearlist_8.get(i))))
    sample8 = ls8_x[ls8_x['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)

year_extract = pd.concat([l5_extract,l7_extract,l8_extract])

year_extract

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,...,QA_PIXEL,ST_CDIST,flats,SR_B1,ST_QA,ST_DRAD,ndvi,SR_QA_AEROSOL,SR_B6,ST_B10
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,...,5463.272727,375.818182,0.025739,0.047382,409.454545,1095.000000,0.327692,NaN,NaN,NaN
19,2,A1,2000,1,1,31.536199,1152.400,21,-81.296118,10/11/2000,...,5455.058824,171.000000,0.023890,0.045548,430.176471,1039.764706,0.360389,NaN,NaN,NaN
21,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,...,5468.444444,129.555556,0.063202,0.051271,438.555556,1025.555556,0.316724,NaN,NaN,NaN
33,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,...,5440.000000,193.333333,0.007976,0.043530,408.238095,1016.571429,0.420151,NaN,NaN,NaN
52,6,A1,2000,1,1,31.388423,1032.632,20,-81.279792,10/12/2000,...,5440.000000,63.000000,0.042608,0.054377,454.000000,1240.384615,0.316812,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154,7,A4,2019,2,8,31.334597,2365.620,14,-81.478039,10/8/2019,...,21824.000000,100.933333,0.015959,0.015950,408.733333,1338.400000,0.536799,160.266667,0.091179,298.036838
3160,7,A4,2019,1,12,31.335017,1821.010,12,-81.478711,10/8/2019,...,21824.000000,110.466667,0.002758,0.017188,377.933333,1288.066667,0.583338,149.466667,0.111881,297.772511
3164,7,A4,2019,1,13,31.334934,2482.584,14,-81.478709,10/8/2019,...,21824.000000,110.533333,0.010794,0.017430,379.733333,1288.066667,0.515524,143.466667,0.092701,297.890774
3168,7,A4,2019,1,16,31.334682,2511.936,17,-81.478564,10/8/2019,...,21824.000000,110.533333,0.010794,0.017430,379.733333,1288.066667,0.515524,143.466667,0.092701,297.890774


In [15]:
#With loop to cut out unnecessary code:
years_l5 = range(2000, 2012)
years_l7 = range(2012, 2013)
years_l8 = range(2013, 2021)

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(peaklist_5.get(i))))
    sample5 = ls5_x[ls5_x['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(peaklist_7.get(i))))
    sample7 = ls7_x[ls7_x['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(peaklist_8.get(i))))
    sample8 = ls8_x[ls8_x['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)

peak_extract = pd.concat([l5_extract,l7_extract,l8_extract])

peak_extract

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,...,QA_PIXEL,ST_CDIST,flats,SR_B1,ST_QA,ST_DRAD,ndvi,SR_QA_AEROSOL,SR_B6,ST_B10
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,...,5440.0,166.000000,0.009257,0.043774,388.500000,1605.500000,0.471350,NaN,NaN,NaN
19,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,...,5440.0,381.000000,0.067190,0.045520,306.000000,1512.000000,0.393872,NaN,NaN,NaN
31,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,...,5440.0,333.428571,0.007745,0.038071,390.571429,1391.857143,0.468609,NaN,NaN,NaN
50,6,A1,2000,1,1,31.388423,1032.632,20,-81.279792,10/12/2000,...,5440.0,43.500000,0.049438,0.071617,504.000000,1603.000000,0.316797,NaN,NaN,NaN
69,9,A1,2000,1,1,31.352109,1484.164,20,-81.333734,10/16/2000,...,5440.0,248.666667,0.021537,0.032907,325.666667,1145.666667,0.399392,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2740,7,A4,2019,2,8,31.334597,2365.620,14,-81.478039,10/8/2019,...,21824.0,49.000000,0.002498,0.011963,525.250000,1915.750000,0.651431,200.500000,0.082439,300.123482
2746,7,A4,2019,1,12,31.335017,1821.010,12,-81.478711,10/8/2019,...,21824.0,68.333333,0.000291,0.013904,448.666667,1893.000000,0.688841,202.666667,0.109980,299.338192
2750,7,A4,2019,1,13,31.334934,2482.584,14,-81.478709,10/8/2019,...,21824.0,68.333333,0.001975,0.014234,454.000000,1893.000000,0.617637,202.666667,0.083360,299.453265
2754,7,A4,2019,1,16,31.334682,2511.936,17,-81.478564,10/8/2019,...,21824.0,68.333333,0.001975,0.014234,454.000000,1893.000000,0.617637,202.666667,0.083360,299.453265


In [16]:
#Daymet for loop and elevation
years_dm = range(2000, 2021)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(dm_list.get(i))))
    sampledm = dm_x[dm_x['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
daymet_extract = pd.concat(daymet_list)

daymet_extract

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,swe,tmax,srad,tmin,vp,prcp,dayl
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,0,25.069397,343.573212,14.079397,1765.701416,2.489973,43200.085938
19,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,0,24.539864,336.004730,15.079507,1874.970703,2.319699,43200.085938
31,4,A1,2000,1,1,31.451676,1534.304,25,-81.365498,10/10/2000,0,24.868795,340.215027,14.635425,1827.156250,2.472329,43200.085938
45,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,0,24.818274,340.010864,14.757424,1839.845947,2.445726,43200.085938
64,6,A1,2000,1,1,31.388423,1032.632,20,-81.279792,10/12/2000,0,24.615616,334.624725,15.242548,1893.765625,2.393671,43200.085938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869,7,A4,2019,2,8,31.334597,2365.620,14,-81.478039,10/8/2019,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938
2875,7,A4,2019,1,12,31.335017,1821.010,12,-81.478711,10/8/2019,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938
2879,7,A4,2019,1,13,31.334934,2482.584,14,-81.478709,10/8/2019,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938
2883,7,A4,2019,1,16,31.334682,2511.936,17,-81.478564,10/8/2019,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938


In [18]:
#Daymet peak time
years_dm = range(2000, 2021)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(peakdm_list.get(i))))
    sampledm = dm_x[dm_x['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
peakdaymet_extract = pd.concat(daymet_list)

peakdaymet_extract

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,swe,tmax,srad,tmin,vp,prcp,dayl
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,0,28.971413,346.619019,18.999348,2280.965332,3.261631,43642.554688
19,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,0,28.514999,337.377716,19.984131,2415.145996,2.840109,43642.554688
31,4,A1,2000,1,1,31.451676,1534.304,25,-81.365498,10/10/2000,0,28.807716,342.130432,19.541521,2353.217041,3.208369,43640.546875
45,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,0,28.765326,341.652161,19.660870,2369.350830,3.210978,43640.546875
64,6,A1,2000,1,1,31.388423,1032.632,20,-81.279792,10/12/2000,0,28.587391,333.892059,20.133043,2434.780029,3.313043,43639.546875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869,7,A4,2019,2,8,31.334597,2365.620,14,-81.478039,10/8/2019,0,31.074240,326.310333,21.603912,2614.622314,5.006957,43747.886719
2875,7,A4,2019,1,12,31.335017,1821.010,12,-81.478711,10/8/2019,0,31.074240,326.310333,21.603912,2614.622314,5.006957,43747.886719
2879,7,A4,2019,1,13,31.334934,2482.584,14,-81.478709,10/8/2019,0,31.074240,326.310333,21.603912,2614.622314,5.006957,43747.886719
2883,7,A4,2019,1,16,31.334682,2511.936,17,-81.478564,10/8/2019,0,31.074240,326.310333,21.603912,2614.622314,5.006957,43747.886719


In [17]:
#Final steps - extra rows in merge?

#MayOct and elevation
dfx = pd.merge(landsat_extract, dem_vals, how = 'left')

#dfx and AugOct
peak_extract.rename(
    columns={
        'SR_B1':'SR_B1_peak','SR_B2':'SR_B2_peak','SR_B3':'SR_B3_peak','SR_B4':'SR_B4_peak', 'SR_B5':'SR_B5_peak',
        'SR_B6':'SR_B6_peak', 'SR_B7':'SR_B7_peak', 'flats':'flats_peak'
    }, inplace=True
)

dfx1 = pd.merge(dfx, peak_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='outer',
                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx1 and year_extract

year_extract.rename(
    columns={
        'SR_B1':'SR_B1_year','SR_B2':'SR_B2_year','SR_B3':'SR_B3_year','SR_B4':'SR_B4_year', 'SR_B5':'SR_B5_year',
        'SR_B6':'SR_B6_year', 'SR_B7':'SR_B7_year', 'flats':'flats_year'
    }, inplace=True
)

dfx2 = pd.merge(dfx1, year_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='outer',
                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx2 and daymet mayoct
dfx3 = pd.merge(dfx2, daymet_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right')

#dfx3 and daymet peak
peakdaymet_extract.rename(
    columns={
        'swe':'swe_peak','tmax':'tmax_peak','tmin':'tmin_peak','srad':'srad_peak', 'vp':'vp_peak',
        'prcp':'prcp_peak', 'dayl':'dayl_peak'
    }, inplace=True
)

df = pd.merge(dfx3, peakdaymet_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

display(df)

NameError: name 'peakdaymet_extract' is not defined

In [18]:
for col in daymet_extract.columns:
    print(col)
    
    ##EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'testx.csv')
# df.to_csv(out_csv, index = False)

Site
Species_Code
Year
Zone
Plot
Latitude
Plant_Biomass
Sample_Size
Longitude
Date
swe
tmax
srad
tmin
vp
prcp
dayl


In [19]:
##Bands and indices
df['Sensor'] = np.where(df['Year']<2013, 'Landsat 5', 'Landsat 8') ##make sure no other sensors are being used

df.loc[df['Year'] == 2012, 'Sensor'] = 'Landsat 7'

df['ndvi'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5']-df['SR_B4'])/(df['SR_B5']+df['SR_B4']), \
                      (df['SR_B4']-df['SR_B3'])/(df['SR_B4']+df['SR_B3'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2'], df['SR_B1'])
df['Green_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3'], df['SR_B2'])
df['Red_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4'], df['SR_B3'])
df['NIR_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5'], df['SR_B4'])
df['SWIR1_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6'], df['SR_B5'])
df['SWIR2_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7'], df['SR_B7'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi'] = ((df['NIR_band']-df['Red_band'])*1.5)/(df['NIR_band']+df['Red_band']+0.5)
df['wdrvi5'] = (0.5*df['NIR_band']-df['Red_band'])/(0.5*df['NIR_band']+df['Red_band'])
df['nd_r_g'] = (df['Red_band']-df['Green_band'])/(df['Red_band']+df['Green_band'])
df['nd_g_b'] = (df['Green_band']-df['Blue_band'])/(df['Green_band']+df['Blue_band'])
df['nd_swir2_nir'] = (df['SWIR2_band']-df['NIR_band'])/(df['SWIR2_band']+df['NIR_band'])
df['nd_swir2_r'] = (df['SWIR2_band']-df['Red_band'])/(df['SWIR2_band']+df['Red_band'])

display(df)

##EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'df_mayoct.csv')
# df.to_csv(out_csv, index = False)

,Site,Species_Code_x,Year,Zone,Plot,Latitude_x,Plant_Biomass,Sample_Size_x,Longitude_x,Date_x,...,Red_band,NIR_band,SWIR1_band,SWIR2_band,savi,wdrvi5,nd_r_g,nd_g_b,nd_swir2_nir,nd_swir2_r
0,1,A1,2000,1,1,31.538627,516.324,23.0,-81.422539,10/11/2000,...,0.061392,0.128653,0.101340,0.059678,0.146209,0.023339,-0.026315,0.111473,-0.366242,-0.014158
1,3,A1,2000,1,1,31.518920,710.396,20.0,-81.228956,10/18/2000,...,0.066349,0.132453,0.090460,0.052450,0.141895,-0.000920,-0.057429,0.150614,-0.432676,-0.116992
2,4,NaN,2000,1,1,NaN,1534.304,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,A1,2000,1,1,31.436090,2022.264,25.0,-81.339919,10/10/2000,...,0.058874,0.151791,0.111497,0.063016,0.196118,0.126296,-0.062060,0.209653,-0.413275,0.033981
4,6,A1,2000,1,1,31.388423,1032.632,20.0,-81.279792,10/12/2000,...,0.072143,0.142254,0.106089,0.061242,0.147211,-0.007090,-0.028964,0.138535,-0.398103,-0.081726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,7,A4,2019,2,8,31.334597,2365.620,14.0,-81.478039,10/8/2019,...,0.038250,0.167424,0.079929,0.037363,0.274577,0.372759,-0.100011,0.408297,-0.635105,-0.011729
2908,7,A4,2019,1,12,31.335017,1821.010,12.0,-81.478711,10/8/2019,...,0.040267,0.208028,0.110234,0.052618,0.336285,0.441820,-0.087693,0.380374,-0.596246,0.132969
2909,7,A4,2019,1,13,31.334934,2482.584,14.0,-81.478709,10/8/2019,...,0.039243,0.166458,0.086199,0.042120,0.270401,0.359150,-0.080947,0.359139,-0.596120,0.035362
2910,7,A4,2019,1,16,31.334682,2511.936,17.0,-81.478564,10/8/2019,...,0.039243,0.166458,0.086199,0.042120,0.270401,0.359150,-0.080947,0.359139,-0.596120,0.035362


In [20]:
df['ndvi_year'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5_year']-df['SR_B4_year'])/(df['SR_B5_year']+df['SR_B4_year']), \
                      (df['SR_B4_year']-df['SR_B3_year'])/(df['SR_B4_year']+df['SR_B3_year'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2_year'], df['SR_B1_year'])
df['Green_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3_year'], df['SR_B2_year'])
df['Red_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4_year'], df['SR_B3_year'])
df['NIR_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5_year'], df['SR_B4_year'])
df['SWIR1_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6_year'], df['SR_B5_year'])
df['SWIR2_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7_year'], df['SR_B7_year'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi_year'] = ((df['NIR_band_year']-df['Red_band_year'])*1.5)/(df['NIR_band_year']+df['Red_band_year']+0.5)
df['wdrvi5_year'] = (0.5*df['NIR_band_year']-df['Red_band_year'])/(0.5*df['NIR_band_year']+df['Red_band_year'])
df['nd_r_g_year'] = (df['Red_band_year']-df['Green_band_year'])/(df['Red_band_year']+df['Green_band_year'])
df['nd_g_b_year'] = (df['Green_band_year']-df['Blue_band_year'])/(df['Green_band_year']+df['Blue_band_year'])
df['nd_swir2_nir_year'] = (df['SWIR2_band_year']-df['NIR_band_year'])/(df['SWIR2_band_year']+df['NIR_band_year'])
df['nd_swir2_r_year'] = (df['SWIR2_band_year']-df['Red_band_year'])/(df['SWIR2_band_year']+df['Red_band_year'])

In [21]:
df['ndvi_peak'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5_peak']-df['SR_B4_peak'])/(df['SR_B5_peak']+df['SR_B4_peak']), \
                      (df['SR_B4_peak']-df['SR_B3_peak'])/(df['SR_B4_peak']+df['SR_B3_peak'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2_peak'], df['SR_B1_peak'])
df['Green_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3_peak'], df['SR_B2_peak'])
df['Red_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4_peak'], df['SR_B3_peak'])
df['NIR_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5_peak'], df['SR_B4_peak'])
df['SWIR1_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6_peak'], df['SR_B5_peak'])
df['SWIR2_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7_peak'], df['SR_B7_peak'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi_peak'] = ((df['NIR_band_peak']-df['Red_band_peak'])*1.5)/(df['NIR_band_peak']+df['Red_band_peak']+0.5)
df['wdrvi5_peak'] = (0.5*df['NIR_band_peak']-df['Red_band_peak'])/(0.5*df['NIR_band_peak']+df['Red_band_peak'])
df['nd_r_g_peak'] = (df['Red_band_peak']-df['Green_band_peak'])/(df['Red_band_peak']+df['Green_band_peak'])
df['nd_g_b_peak'] = (df['Green_band_peak']-df['Blue_band_peak'])/(df['Green_band_peak']+df['Blue_band_peak'])
df['nd_swir2_nir_peak'] = (df['SWIR2_band_peak']-df['NIR_band_peak'])/(df['SWIR2_band_peak']+df['NIR_band_peak'])
df['nd_swir2_r_peak'] = (df['SWIR2_band_peak']-df['Red_band_peak'])/(df['SWIR2_band_peak']+df['Red_band_peak'])

In [22]:
##https://stackoverflow.com/questions/18557860/how-to-create-a-list-with-a-range-of-years

##WITH VARIABLE YEAR IN FILE NAME

df = df[df['Red_band'].notna()]
df = df[df['SR_B4_peak'].notna()]
df = df[df['SR_B4_year'].notna()]
df

,Site,Species_Code_x,Year,Zone,Plot,Latitude_x,Plant_Biomass,Sample_Size_x,Longitude_x,Date_x,...,Red_band_peak,NIR_band_peak,SWIR1_band_peak,SWIR2_band_peak,savi_peak,wdrvi5_peak,nd_r_g_peak,nd_g_b_peak,nd_swir2_nir_peak,nd_swir2_r_peak
0,1,A1,2000,1,1,31.538627,516.324,23.0,-81.422539,10/11/2000,...,0.038274,0.106460,0.067589,0.042976,0.158638,0.163450,-0.155874,0.089777,-0.424822,0.057877
1,3,A1,2000,1,1,31.518920,710.396,20.0,-81.228956,10/18/2000,...,0.048985,0.112648,0.064468,0.041148,0.144331,0.069688,-0.114135,0.150172,-0.464905,-0.086955
3,5,A1,2000,1,1,31.436090,2022.264,25.0,-81.339919,10/10/2000,...,0.049307,0.136132,0.094234,0.050792,0.190007,0.159824,-0.088394,0.214542,-0.456550,0.014835
4,6,A1,2000,1,1,31.388423,1032.632,20.0,-81.279792,10/12/2000,...,0.086110,0.159576,0.114449,0.064344,0.147782,-0.038107,-0.033714,0.125209,-0.425297,-0.144671
5,9,A1,2000,1,1,31.352109,1484.164,20.0,-81.333734,10/16/2000,...,0.043641,0.101895,0.079070,0.042678,0.135362,0.077247,-0.043305,0.182426,-0.409596,-0.011150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,7,A4,2019,2,8,31.334597,2365.620,14.0,-81.478039,10/8/2019,...,0.038549,0.184890,0.082439,0.038432,0.303428,0.411441,-0.093451,0.417960,-0.655816,-0.001518
2908,7,A4,2019,1,12,31.335017,1821.010,12.0,-81.478711,10/8/2019,...,0.040753,0.221318,0.109980,0.052331,0.355409,0.461691,-0.082167,0.411056,-0.617533,0.124377
2909,7,A4,2019,1,13,31.334934,2482.584,14.0,-81.478709,10/8/2019,...,0.037371,0.156748,0.083360,0.041129,0.257976,0.354256,-0.065875,0.366045,-0.584297,0.047877
2910,7,A4,2019,1,16,31.334682,2511.936,17.0,-81.478564,10/8/2019,...,0.037371,0.156748,0.083360,0.041129,0.257976,0.354256,-0.065875,0.366045,-0.584297,0.047877


In [43]:
##EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'df_testx.csv')
# df.to_csv(out_csv, index = False)